In [23]:
import csv
import json
import sqlite3
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime
import torch
import pickle
import random
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import os
from torchvision import models
from torch import nn as nn
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from PIL import Image
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn import svm
from copy import deepcopy
with open('color_to_bert.p','rb') as F:
    color_to_embedding = pickle.load(F)
    color_to_embedding['none'] = color_to_embedding['white']

In [34]:
def plot_learning_curve_single_subject(marks_conf,counterbalance):

    fig, ax = plt.subplots(figsize=(5, 3))
    rect = patches.Rectangle((0,-0.2), 3, 1.4, linewidth=1,  facecolor='yellow', alpha=0.5)#edgecolor='r',
    conf_mat = marks_conf[9]
    marks_acc = [(conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()]
    for i in range(1,6):
        conf_mat = marks_conf[(i+1)*8-1] - marks_conf[i*8-1]
        acc = (conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()
        marks_acc.append(acc)

    ax.scatter(range(len(marks_acc)),marks_acc,c='black',s=60)

    for i in range(len(marks_acc)-1):
        ax.plot([i,i+1],[marks_acc[i],marks_acc[i+1]],c='black')
    plt.yticks([_*0.1 for _ in range(0,11,2)],[round(_*0.1,1) for _ in range(0,11,2)],fontsize=20)
    plt.xticks(range(len(marks_acc)),[_*8 for _ in range(1,len(marks_acc)+1)],fontsize=20)
    plt.suptitle('counterbalance: {}'.format(counterbalance), fontsize=20)
    # Add the patch to the Axes
    ax.add_patch(rect)
    ax.xaxis.set_ticks_position('none') 
    ax.yaxis.set_ticks_position('none') 
    ax.spines[['right', 'left']].set_visible(False)

def construct_confmat():
    labels = ['h_azure','h_histogram','h_google']#,'h_catch']
    confmatrices = {**{'h_machine':np.zeros((2,2))}, **{label: np.zeros((1,2)) for label in labels}}
    return confmatrices

def print_metadata(data,dummy_mark='NA'):
    print("====="*20)
    datetime_object1 = datetime.strptime(data[11].split('.')[0], '%Y-%m-%d %H:%M:%S')
    datetime_object2 = datetime.strptime(data[13].split('.')[0], '%Y-%m-%d %H:%M:%S')
    time_used = datetime_object2 - datetime_object1
    time_used = round(time_used.seconds/60,1)
    print("[Finished Date]",datetime_object2)
    print("[Time Used] {} mins".format(time_used))
    print("[Questionnaire]", json.loads(data[-1])["questiondata"])
    print("[Counterbalance]", json.loads(data[-1])["counterbalance"])
    print("[Dummy Test Acc]", dummy_mark)
    return time_used, json.loads(data[-1])["counterbalance"]

def get_gt_from_pagename(pagename):
    # example:  'page_name': '568_e_0.html/',
    return {'a':'Human','b':'Machine','c':'Machine','d':'Machine','e':'Catch'}[pagename.split('_')[1]]

def get_model_from_pagename(pagename):
    # example:  'page_name': '568_e_0.html/',
    return {'a':'','b':'azure','c':'histogram','d':'google'}[pagename.split('_')[1]]


def image_imbedding_from_pagename(pagename):
    root = "/home/liuxiao/TuringGithub/dominant_color_recognition/MturkExps/dominant_color_recognition/image_embeddings"
    imgid = pagename.split('_')[0]+'.p'
    with open(os.path.join(root,imgid),'rb') as f:
        imb = pickle.load(f)
    return imb 

def sample_to_imbedding(pagename,gt):
    with open("/home/liuxiao/TuringGithubRunnable/dominant_color_recognition/Feedback_Color/static/responses_color/{}".format(trial_data['trialData']['page_name'][:-1]), 'r') as f:
        raw_text = f.readlines()[0]
    color = raw_text.split('Speaker Response: ')[-1]
    color = color.split('</h4>')[0]
    color_imbbedding = color_to_embedding[color.lower()]
    img_imbbedding = image_imbedding_from_pagename(pagename)
    return np.array(torch.hstack((img_imbbedding, color_imbbedding[0])).detach()).flatten(), {'human':0,'machine':1}[gt.lower()]
    
conn = sqlite3.connect("ec2_db/turing_feedback_color.db")
cursor = conn.cursor()
cursor.execute("select * from tablechairDB;")

label_dict = {'human':0,'machine':1,'Human':0,'Machine':1,'azure':1,'histogram':1,'google':1,'catch':1}
oneWord_curr_d = {'0':True,'1':False} # dummy test gt
topic_d = {'1':True,'2':False} # user response to dummy tests
modelname_dict = {'catch':'CatchTrial','azure':'Azure','histogram':'Histogram','google':'Google'}

count = 0
participant_count = 0
trials_dict = {}
all_data = cursor.fetchall()
completed_counterbalances = []


confmat_overall = construct_confmat()
catch_trial_mat_overall = np.zeros((1,2))
swap_trial_mat_overall = np.zeros((1,2))
confmat_overall_AI = construct_confmat()

ai_judge = {}
trialnum = 0
for i in range(len(all_data)):
    if all_data[i][15] in [5]:
#         print(all_data[i][11])
        data_dict = json.loads(all_data[i][17])
        if data_dict['mode'] != 'live':
            continue
        # filter out subjects using dummy trial accuracy
        # original threshold in the paper: 7/12
        dummy_mark = 0
        nTrials = 100
        threshold = nTrials*(7/12) # change to number of total trials 
        for dat in data_dict['data']:

            if dat['trialdata']['phase'] == 'TEST':
                present = dat['trialdata']['oneWord_curr'] == '0'
                response_present = dat['trialdata']['topic'] == '1'
                if present == response_present:
                    dummy_mark += 1
        if dummy_mark < threshold:
            continue
        
        
        # construct conf matrices
        time_used, counterbalance = print_metadata(all_data[i],"{}/{}".format(dummy_mark,nTrials))
        completed_counterbalances.append(counterbalance)
#         marks = []
#         marks_conf = []
        confmatrices = construct_confmat() # without catch
        confmat_AI = construct_confmat()
        catch_trial_mat = np.zeros((1,2))
        swap_trial_mat = np.zeros((1,2))
        marks = []
        marks_conf = []
        participant_count += 1
        nTrial = 0
        
        X_train, y_train, X_test, y_test = [], [], [], []
        test_samples = []
        for dat in data_dict['data']:
            if dat['trialdata']['phase'] == 'TEST':
                
                nTrial+=1
                trial_data = {}
                trial_data['workerID'] = data_dict['workerId']
                trial_data['workerData'] = data_dict['questiondata']
                trial_data['trialData'] = dat['trialdata']

                
#                 trials_dict['Trial_'+str(count)] = trial_data
#                 trial_data = trial_data['trialData']
                trial_data['trialData']['correct response'] = get_gt_from_pagename(trial_data['trialData']['page_name'])
                trial_data['trialData']['choose response'] = trial_data['trialData']['Aclass']
                
#                 print(trial_data)
                if trial_data['trialData']['correct response'] == 'Catch':
                    catch_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    catch_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1
                elif trial_data['trialData']['correct response'] == 'Swapped':
                    swap_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    swap_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1
                elif nTrial > 40:  
                    confmatrices['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    confmat_overall['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    if trial_data['trialData']['correct response'] == "Machine":
                        model = get_model_from_pagename(trial_data['trialData']['page_name'])
                        confmatrices['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
                        confmat_overall['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
                    marks.append(1 if trial_data['trialData']['correct response'] == trial_data['trialData']['choose response'] else 0)
                    marks_conf.append(deepcopy(confmatrices['h_machine']))
                    
                    ### === Add for AI === ### 
                    x,y = sample_to_imbedding(trial_data['trialData']['page_name'],trial_data['trialData']['correct response'])
                    X_test.append(x)
                    y_test.append(y)
                    trial_data['trialData']["machine_groundtruth"] = get_model_from_pagename(trial_data['trialData']['page_name'])
                    test_samples.append(deepcopy(trial_data['trialData']))
                    
                elif nTrial <= 40:
                    ### === Add for AI === ###
                    x,y = sample_to_imbedding(trial_data['trialData']['page_name'],trial_data['trialData']['correct response'])
                    X_train.append(x)
                    y_train.append(y)
                    
#                 trial_data['trialData'] = dat['trialdata']
                if trial_data['trialData']['correct response']  in ['Catch','Swapped'] or nTrial <=40:
                    continue
                
                count += 1
        print('Human Judge',confmatrices)
        print('catch',catch_trial_mat)                
        ### ===================== AI judge ======================== ###
#         assert len(X_train) == 40 and len(y_train) == 40 and len(X_test) == 40 and len(y_test) == 40
        clf = svm.SVC(verbose=True)
        clf.fit(list(X_train), y_train)
        predictions = clf.predict(X_test)
        pred_indx = 0
                
        for sample in test_samples:
            
            trialnum+=1
            response_speaker = ''
            if(predictions[pred_indx]==0):
                response_speaker = "human"
            elif(predictions[pred_indx]==1):
                response_speaker = "machine"

            confmat_AI['h_machine'][label_dict[sample['correct response']]][predictions[pred_indx]] += 1
            confmat_overall_AI['h_machine'][label_dict[sample['correct response']]][predictions[pred_indx]] += 1

            if sample['correct response'].lower() == 'machine':  
                model = sample["machine_groundtruth"]
                confmat_AI['h_{}'.format(model)][0][predictions[pred_indx]] += 1
                confmat_overall_AI['h_{}'.format(model)][0][predictions[pred_indx]] += 1

            trialdata = {"response_speaker": response_speaker, "groundtruth":sample["correct response"].lower(), "machine_groundtruth":sample["machine_groundtruth"]}
            ai_judge['Trial_Color_'+str(trialnum)] = {"trialData": trialdata}
            pred_indx+=1  

#         plot_learning_curve_single_subject(marks_conf,participant_count)
        print('AI Judge',confmat_AI)

#         print('swap',swap_trial_mat)
#     else:
#         print(all_data[i][15])
conn.close()

[Finished Date] 2025-03-28 11:04:57
[Time Used] 21.9 mins
[Questionnaire] {'country': 'United States', 'age': '51', 'engagement': '6', 'difficulty': '8', 'mode': 'live', 'gender': 'male', 'education': 'bachelor', 'ai-experience': '1', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 93/100
Human Judge {'h_machine': array([[20.,  9.],
       [13., 13.]]), 'h_azure': array([[4., 3.]]), 'h_histogram': array([[6., 7.]]), 'h_google': array([[3., 3.]])}
catch [[0. 5.]]
[LibSVM]*
optimization finished, #iter = 36
obj = -27.019329, rho = 0.219251
nSV = 40, nBSV = 27
Total nSV = 40
AI Judge {'h_machine': array([[ 1., 28.],
       [ 0., 26.]]), 'h_azure': array([[0., 7.]]), 'h_histogram': array([[ 0., 13.]]), 'h_google': array([[0., 6.]])}
[Finished Date] 2025-03-28 10:52:55
[Time Used] 9.5 mins
[Questionnaire] {'country': 'United States', 'age': '36-40', 'engagement': '9', 'difficulty': '10', 'mode': 'live', 'gender': 'female', 'education': 'high school', 'ai-experience': '1

Human Judge {'h_machine': array([[27., 11.],
       [11.,  5.]]), 'h_azure': array([[2., 1.]]), 'h_histogram': array([[3., 0.]]), 'h_google': array([[6., 4.]])}
catch [[4. 2.]]
[LibSVM]*
optimization finished, #iter = 39
obj = -27.363314, rho = 0.598959
nSV = 40, nBSV = 27
Total nSV = 40
AI Judge {'h_machine': array([[ 1., 37.],
       [ 0., 16.]]), 'h_azure': array([[0., 3.]]), 'h_histogram': array([[0., 3.]]), 'h_google': array([[ 0., 10.]])}
[Finished Date] 2025-03-28 11:13:40
[Time Used] 17.5 mins
[Questionnaire] {'country': 'United States', 'age': '45-50', 'engagement': '7', 'difficulty': '5', 'mode': 'live', 'gender': 'female', 'education': 'high school', 'ai-experience': '1', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 91/100
Human Judge {'h_machine': array([[16., 19.],
       [ 4., 14.]]), 'h_azure': array([[1., 3.]]), 'h_histogram': array([[ 2., 11.]]), 'h_google': array([[1., 0.]])}
catch [[4. 3.]]
[LibSVM]*
optimization finished, #iter = 30
obj = -27

[LibSVM]*.*
optimization finished, #iter = 45
obj = -26.253362, rho = -0.587331
nSV = 40, nBSV = 21
Total nSV = 40
AI Judge {'h_machine': array([[30.,  0.],
       [26.,  0.]]), 'h_azure': array([[13.,  0.]]), 'h_histogram': array([[7., 0.]]), 'h_google': array([[6., 0.]])}
[Finished Date] 2025-03-28 11:46:29
[Time Used] 14.5 mins
[Questionnaire] {'country': 'United States', 'age': '31-35', 'engagement': '4', 'difficulty': '6', 'mode': 'live', 'gender': 'male', 'education': 'high school', 'ai-experience': '2', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 89/100
Human Judge {'h_machine': array([[22.,  8.],
       [ 9., 14.]]), 'h_azure': array([[2., 7.]]), 'h_histogram': array([[3., 3.]]), 'h_google': array([[4., 4.]])}
catch [[0. 7.]]
[LibSVM]*.*
optimization finished, #iter = 40
obj = -25.270212, rho = -0.646629
nSV = 40, nBSV = 23
Total nSV = 40
AI Judge {'h_machine': array([[30.,  0.],
       [23.,  0.]]), 'h_azure': array([[9., 0.]]), 'h_histogram': array([[

[Dummy Test Acc] 89/100
Human Judge {'h_machine': array([[16., 16.],
       [ 4., 14.]]), 'h_azure': array([[2., 9.]]), 'h_histogram': array([[1., 2.]]), 'h_google': array([[1., 3.]])}
catch [[10.  0.]]
[LibSVM]*.*
optimization finished, #iter = 47
obj = -27.694023, rho = 0.360013
nSV = 40, nBSV = 24
Total nSV = 40
AI Judge {'h_machine': array([[ 0., 32.],
       [ 0., 18.]]), 'h_azure': array([[ 0., 11.]]), 'h_histogram': array([[0., 3.]]), 'h_google': array([[0., 4.]])}
[Finished Date] 2025-03-28 13:17:17
[Time Used] 40.2 mins
[Questionnaire] {'country': 'United States', 'age': '36-40', 'engagement': '10', 'difficulty': '0', 'mode': 'live', 'gender': 'male', 'education': 'high school', 'ai-experience': '1', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 89/100
Human Judge {'h_machine': array([[10., 20.],
       [11., 10.]]), 'h_azure': array([[5., 0.]]), 'h_histogram': array([[5., 7.]]), 'h_google': array([[1., 3.]])}
catch [[5. 4.]]
[LibSVM]*.*
optimization fin

Human Judge {'h_machine': array([[11., 14.],
       [ 5., 25.]]), 'h_azure': array([[1., 2.]]), 'h_histogram': array([[ 3., 14.]]), 'h_google': array([[1., 9.]])}
catch [[0. 5.]]
[LibSVM]*.*
optimization finished, #iter = 46
obj = -25.156184, rho = -0.591901
nSV = 40, nBSV = 21
Total nSV = 40
AI Judge {'h_machine': array([[25.,  0.],
       [30.,  0.]]), 'h_azure': array([[3., 0.]]), 'h_histogram': array([[17.,  0.]]), 'h_google': array([[10.,  0.]])}
[Finished Date] 2025-03-28 13:34:24
[Time Used] 15.9 mins
[Questionnaire] {'country': 'United States', 'age': '41-45', 'engagement': '9', 'difficulty': '8', 'mode': 'live', 'gender': 'male', 'education': 'master', 'ai-experience': '1', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 91/100
Human Judge {'h_machine': array([[25.,  6.],
       [ 8., 14.]]), 'h_azure': array([[2., 2.]]), 'h_histogram': array([[3., 5.]]), 'h_google': array([[3., 7.]])}
catch [[0. 7.]]
[LibSVM]*
optimization finished, #iter = 20
obj = -29.5

In [56]:
with open("/home/liuxiao/TuringGithubRunnable/dominant_color_recognition/Plot/Data_all/humanjudge_Color_Feedback_testphase.json", 'w') as f:
    json.dump(trials_dict,f)

In [35]:
json.dump(ai_judge, open("/home/liuxiao/TuringGithubRunnable/dominant_color_recognition/Plot/Data_all/ai_judge_feedback_color.json","w"))
json.dump(ai_judge, open("/home/liuxiao/TuringGithub/XiaoData/all_results/ai_judge_feedback_color.json","w"))


In [36]:
confmat_overall_AI

{'h_machine': array([[856., 628.],
        [686., 489.]]),
 'h_azure': array([[238., 171.]]),
 'h_histogram': array([[231., 170.]]),
 'h_google': array([[217., 148.]])}

In [40]:
856/(856+628),686/(686.+ 489),231/(231.+ 170),217/(217.+148)

(0.5768194070080862,
 0.5838297872340426,
 0.5760598503740648,
 0.5945205479452055)

In [52]:
confmat_overall

{'h_machine': array([[867., 617.],
        [505., 670.]]),
 'h_azure': array([[172., 237.]]),
 'h_histogram': array([[154., 247.]]),
 'h_google': array([[179., 186.]])}

In [57]:
trials_dict

{'Trial_Color_1': {'workerID': 'A2JPOXYZM5AJZZ',
  'workerData': {'country': 'United States',
   'age': '51',
   'engagement': '6',
   'difficulty': '8',
   'mode': 'live',
   'gender': 'male',
   'education': 'bachelor',
   'ai-experience': '1',
   'condition': 0,
   'native': 'yes'},
  'trialData': {'rt': 55480,
   'Aclass': 'Human',
   'hit': '97',
   'imageID': 'https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/1150.jpg',
   'trial': 41,
   'page_name': '1150_a_0.html/',
   'phase': 'TEST',
   'response_gender': 'NA',
   'response_speaker': 'human',
   'response_object': '2',
   'groundtruth': 'human',
   'object_groundtruth': '1',
   'machine_groundtruth': '',
   'imgtype': 'naturaldesign'}},
 'Trial_Color_2': {'workerID': 'A2JPOXYZM5AJZZ',
  'workerData': {'country': 'United States',
   'age': '51',
   'engagement': '6',
   'difficulty': '8',
   'mode': 'live',
   'gender': 'male',
   'education': 'bachelor',
   'ai-experience': '1',
   'condition': 0,
   'native': 'yes'},
